# VolPulse — Minimal Example
This notebook shows how to run the **VolPulse** pipeline on copper prices, inspect outputs, and plot quick sanity charts.

**Assumptions**
- Your repo has the folder `C:\\Code\\Metals\\Docs\\Copper\\vol\\` containing `vol_pulse.py` and `vol_pulse.yaml`.
- You have a CSV of copper prices with columns: `Date` and `Price`, sorted by date ascending.
If you don't have a CSV, the notebook will generate a synthetic demo series.


## 0) Setup

In [ ]:
# If needed, uncomment:
# !pip install pandas numpy pyyaml matplotlib

import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

vol_path = r"C:\Code\Metals\Docs\Copper\vol"
if os.path.isdir(vol_path) and vol_path not in sys.path:
    sys.path.insert(0, vol_path)

from vol_pulse import compute_vol_pulse
print("Imported compute_vol_pulse from:", vol_path)

## 1) Load price data

In [ ]:
csv_path = r"C:\Code\Metals\Data\copper_prices.csv"  # change to your path

def load_or_make_demo(csv_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, parse_dates=["Date"])
        df = df.sort_values("Date").reset_index(drop=True)
        return df
    dates = pd.bdate_range("2020-01-01", periods=800)
    price = 6000 + np.cumsum(np.random.normal(0, 20, size=len(dates))) + np.linspace(0, 400, len(dates))
    return pd.DataFrame({"Date": dates, "Price": price})

df_prices = load_or_make_demo(csv_path)
df_prices.head()

## 2) Run VolPulse

In [ ]:
config_path = os.path.join(vol_path, "vol_pulse.yaml")
pulse = compute_vol_pulse(df_prices, config_path=config_path)
print(pulse.tail(3))
pulse.to_csv("copper_vol_pulse_output.csv", index=False)
print("Saved: copper_vol_pulse_output.csv")

## 3) Quick sanity plots

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(pulse['Date'], pulse['rv21'])
plt.title("RV21 (Annualised)"); plt.xlabel("Date"); plt.ylabel("Vol (ann.)")
plt.show()

plt.figure(figsize=(10,4))
plt.plot(pulse['Date'], pulse['pctl'])
plt.title("Vol Percentile (0–1)"); plt.xlabel("Date"); plt.ylabel("Percentile")
plt.show()

plt.figure(figsize=(10,3))
plt.plot(pulse['Date'], pulse['crisis'], drawstyle='steps-post')
if 'high' in pulse.columns:
    plt.plot(pulse['Date'], pulse['high'], drawstyle='steps-post')
plt.title("Crisis (and High) Flags"); plt.xlabel("Date"); plt.ylabel("Flag")
plt.show()

## 4) Merge into sleeves

- Merge `state`, `crisis`, `size_mult`, `allow_new`, `stop_mult` on `Date`.
- Keep helpers (`rv21`, `pctl`, `drv5`, `spike_*`) for reporting/QA.
- A/B test with and without `VolPulse` gating/sizing.
